Set Up

In [126]:
import pandas as pd
import numpy as np
import re
import sqlite3

Files: all three files, country, datadict, indicators

In [ ]:
#switch between the files 

#df = pd.read_excel('SDG_DATA_NATIONAL.xlsx') 
#df = pd.read_csv(r'C:\Users\Norah\Downloads\OPRI (1)\OPRI_DATA_NATIONAL.csv', low_memory=False)
df = pd.read_excel('SCN-SDG_DATA_NATIONAL.xlsx') #EXPGDP.TOT
country = pd.read_excel('UIS UNESCO Country.xlsx', sheet_name='Country') 
datadict = pd.read_excel('UIS UNESCO Country.xlsx', sheet_name='DataDict')
#remember to switch sheets here too
indicators = pd.read_excel('UIS UNESCO Country.xlsx', sheet_name='SCN-SDG')
df

,INDICATOR_ID,COUNTRY_ID,YEAR,VALUE,MAGNITUDE,QUALIFIER
0,EXPGDP.TOT,AGO,2016,0.03229,NaN,NaN
1,EXPGDP.TOT,ALB,2007,0.08757,NaN,NaN
2,EXPGDP.TOT,ALB,2008,0.15412,NaN,NaN
3,EXPGDP.TOT,ARE,2011,0.47252,NaN,NaN
4,EXPGDP.TOT,ARE,2014,0.67523,NaN,NaN
...,...,...,...,...,...,...
4250,RESDEN.INHAB.TFTE,ZMB,1997,54.29949,NaN,NaN
4251,RESDEN.INHAB.TFTE,ZMB,1998,56.42220,NaN,NaN
4252,RESDEN.INHAB.TFTE,ZMB,1999,55.79382,NaN,NaN
4253,RESDEN.INHAB.TFTE,ZMB,2008,41.90146,NaN,NaN


In [134]:
formulatables = {
    'EdPriEnrollHeadCountTtl', 'EdPriEnrollHeadcountFem', 'EdSecLowerEnrollHeadcountTotal', 'EdSecLowerEnrollHeadcountFemale',
    'EdSecUpperEnrollHeadcountTotal', 'EdSecUpperEnrollHeadcountFemale', 'EdSecEnrNoTot', 'EdSecEnrNoFem',
    'EdTerEnrollHeadcountTotal', 'EdTerEnrollHeadcountFem', 'EdPriEnrollHeadcountMal', 'EdSecLowerEnrollHeadcountMale',
    'EdSecUpperEnrollHeadCountMale', 'EdSecEnrNoMal', 'EdTerEnrISCED6HdctMale', 'EdTerEnrollHeadcountMale',
    'EdTerEnrISCED6HdctFem', 'EdTerEnrISCED6HdctTot'
}

SDG + SCN-SDG + OPRI TABLES (no calculation with indicators)

In [137]:
def parse_years(year_str):
    if not year_str or pd.isnull(year_str):
        return []
    years = set()
    for part in str(year_str).split(','):
        part = part.strip()
        if '-' in part:
            start, end = part.split('-')
            years.update(range(int(start), int(end) + 1))
        elif part.isdigit():
            years.add(int(part))
    return sorted(years)

conn = sqlite3.connect(r'C:\IFs\RUNFILES\IFsDataImport - Copy (5) - Copy.db')
cursor = conn.cursor()

for _, row in indicators.iterrows():
    indicator_id = str(row['INDICATOR_ID'])
    table_name = str(row['Variable'])
    sql_table_name = f"Series{table_name}"

    dec_places = datadict.loc[datadict['Variable'] == table_name, 'Decimal Places']
    dec_places = int(dec_places.iloc[0]) if not dec_places.empty and pd.notnull(dec_places.iloc[0]) else 4

    df1 = df[df['INDICATOR_ID'] == indicator_id].copy()
    if table_name in formulatables:
        df1['VALUE'] = df1['VALUE'] * 0.000001

    df2 = df1.drop(columns=['INDICATOR_ID', 'MAGNITUDE', 'QUALIFIER'])
    df2 = df2.pivot(index="COUNTRY_ID", columns="YEAR", values="VALUE")
    #to ensure year columns are int bc some issues in OPRI
    df2.columns = [int(float(col)) for col in df2.columns]
    if len(df2.columns) > 0:
        min_year = min(df2.columns)
        max_year = max(df2.columns)
        intended_years = list(range(min_year, max_year + 1))
    else:
        intended_years = []

    dt = country.merge(df2.reset_index(), on='COUNTRY_ID', how='left')
    dt = dt.drop(columns='COUNTRY_ID')
    dt = dt.rename(columns={'IFs Country': 'Country'})

    dt = dt.reindex(columns=['Country', 'FIPS_CODE'] + intended_years)

    if intended_years:
        dt['Earliest'] = dt[intended_years].bfill(axis=1).iloc[:, 0]
        dt['MostRecent'] = dt[intended_years].ffill(axis=1).iloc[:, -1]
        dt[intended_years + ['Earliest', 'MostRecent']] = dt[intended_years + ['Earliest', 'MostRecent']].round(dec_places)
    else:
        dt['Earliest'] = np.nan
        dt['MostRecent'] = np.nan

    columns_sql = ['Country VARCHAR(255)', 'FIPS_CODE VARCHAR(255)']
    columns_sql += [f'"{col}" DOUBLE(53)' for col in intended_years + ['Earliest', 'MostRecent']]
    create_table_sql = f"CREATE TABLE '{sql_table_name}' ({', '.join(columns_sql)})"
    print("CREATE TABLE SQL:", create_table_sql)

    cursor.execute(f'DROP TABLE IF EXISTS \"{sql_table_name}\"')
    cursor.execute(create_table_sql)

    dt = dt[['Country', 'FIPS_CODE'] + intended_years + ['Earliest', 'MostRecent']]

    print(f"FINAL columns for {sql_table_name}:", dt.columns)
    dt.to_sql(sql_table_name, conn, if_exists='append', index=False)
    print(f"Imported: {sql_table_name}")

conn.commit()
conn.close()

CREATE TABLE SQL: CREATE TABLE 'SeriesR&DGovt%GDP' (Country VARCHAR(255), FIPS_CODE VARCHAR(255), "1996" DOUBLE(53), "1997" DOUBLE(53), "1998" DOUBLE(53), "1999" DOUBLE(53), "2000" DOUBLE(53), "2001" DOUBLE(53), "2002" DOUBLE(53), "2003" DOUBLE(53), "2004" DOUBLE(53), "2005" DOUBLE(53), "2006" DOUBLE(53), "2007" DOUBLE(53), "2008" DOUBLE(53), "2009" DOUBLE(53), "2010" DOUBLE(53), "2011" DOUBLE(53), "2012" DOUBLE(53), "2013" DOUBLE(53), "2014" DOUBLE(53), "2015" DOUBLE(53), "2016" DOUBLE(53), "2017" DOUBLE(53), "2018" DOUBLE(53), "2019" DOUBLE(53), "2020" DOUBLE(53), "2021" DOUBLE(53), "2022" DOUBLE(53), "2023" DOUBLE(53), "Earliest" DOUBLE(53), "MostRecent" DOUBLE(53))
FINAL columns for SeriesR&DGovt%GDP: Index([   'Country',  'FIPS_CODE',         1996,         1997,         1998,
               1999,         2000,         2001,         2002,         2003,
               2004,         2005,         2006,         2007,         2008,
               2009,         2010,         2011,      

Formulas SDG

In [130]:
df = pd.read_csv(r"C:\Users\Norah\Downloads\SDG (2)\SDG_DATA_NATIONAL.csv", low_memory=False)
conn = sqlite3.connect(r'C:\IFs\RUNFILES\IFsDataImport - Copy (5) - Copy.db')
cursor = conn.cursor()

def process_sum_table(ind_ids, table_name, min_year=None, max_year=None):
    g2 = df[df['INDICATOR_ID'] == ind_ids[0]]
    g3 = df[df['INDICATOR_ID'] == ind_ids[1]]
    dec_places = datadict.loc[datadict['Variable'] == table_name, 'Decimal Places']
    dec_places = int(dec_places.iloc[0]) if not dec_places.empty and pd.notnull(dec_places.iloc[0]) else 4

    g2 = g2.drop(columns=['INDICATOR_ID', 'MAGNITUDE', 'QUALIFIER'], errors='ignore')
    g3 = g3.drop(columns=['INDICATOR_ID', 'MAGNITUDE', 'QUALIFIER'], errors='ignore')


    pivot1 = g2.pivot(index='COUNTRY_ID', columns='YEAR', values='VALUE')
    pivot2 = g3.pivot(index='COUNTRY_ID', columns='YEAR', values='VALUE')
    g2g3 = pivot1.add(pivot2, fill_value=0)

    g2g3 = country.merge(g2g3.reset_index(), on='COUNTRY_ID', how='left').drop(columns='COUNTRY_ID')
    g2g3.rename(columns={'IFs Country': 'Country'}, inplace=True)

    all_years = set(pivot1.columns).union(set(pivot2.columns))
    if not all_years:
        print(f"Skipping {table_name}: No data for indicators {ind_ids}")
        return
    if min_year is None:
        min_year = int(min(all_years))
    if max_year is None:
        max_year = int(max(all_years))
    full_years = list(range(min_year, max_year + 1))
    for year in full_years:
        if year not in g2g3.columns:
            g2g3[year] = np.nan
    year_columns = sorted(full_years)
    g2g3 = g2g3[['Country', 'FIPS_CODE'] + year_columns]


    year_columns = [col for col in g2g3.columns if isinstance(col, int) or (isinstance(col, str) and col.isdigit())]
    g2g3['Earliest'] = g2g3[year_columns].bfill(axis=1).iloc[:, 0]
    g2g3['MostRecent'] = g2g3[year_columns].ffill(axis=1).iloc[:, -1]
    g2g3[year_columns + ['Earliest', 'MostRecent']] = g2g3[year_columns + ['Earliest', 'MostRecent']].round(dec_places)

    columns_sql = ['Country VARCHAR(255)', 'FIPS_CODE VARCHAR(255)'] + \
                  [f'"{col}" DOUBLE(53)' for col in year_columns + ['Earliest', 'MostRecent']]
    create_table_sql = f"CREATE TABLE '{table_name}' ({', '.join(columns_sql)})"

    cursor.execute(f'DROP TABLE IF EXISTS "{table_name}"')
    cursor.execute(create_table_sql)
    g2g3.to_sql(table_name, conn, if_exists='append', index=False)

tables_info = [
    (['READ.G2.GPIA', 'READ.G3.GPIA'], 'SeriesEdPri2and3ParityGenderRead'),
    (['READ.G2.LPIA', 'READ.G3.LPIA'], 'SeriesEdPri2and3ParityRurUrbRead'),
    (['READ.G2.WPIA', 'READ.G3.WPIA'], 'SeriesEdPri2and3ParitySocioeconRead'),
    (['MATH.G2.GPIA', 'MATH.G3.GPIA'], 'SeriesEdPri2and3ParityGenderMath'),
    (['MATH.G2.LPIA', 'MATH.G3.LPIA'], 'SeriesEdPri2and3ParityRurUrbMath'),
    (['MATH.G2.WPIA', 'MATH.G3.WPIA'], 'SeriesEdPri2and3ParitySocioeconMath'),
]

for indicators, name in tables_info:
    process_sum_table(indicators, name)



Formulas OPRI

Dividing

In [131]:
df = pd.read_csv(r'C:\Users\Norah\Downloads\OPRI (1)\OPRI_DATA_NATIONAL.csv', low_memory=False)
conn = sqlite3.connect(r'C:\IFs\RUNFILES\IFsDataImport - Copy (5) - Copy.db')
cursor = conn.cursor()

def process_ratio(inop, inri, table_name, min_year=None, max_year=None):
    op = df[df['INDICATOR_ID'] == inop].drop(columns=['INDICATOR_ID', 'MAGNITUDE', 'QUALIFIER'], errors='ignore')
    ri = df[df['INDICATOR_ID'] == inri].drop(columns=['INDICATOR_ID', 'MAGNITUDE', 'QUALIFIER'], errors='ignore')
    
    dec_places = datadict.loc[datadict['Variable'] == table_name, 'Decimal Places']
    dec_places = int(dec_places.iloc[0]) if not dec_places.empty and pd.notnull(dec_places.iloc[0]) else 4

    pivot1 = op.pivot(index='COUNTRY_ID', columns='YEAR', values='VALUE')
    pivot2 = ri.pivot(index='COUNTRY_ID', columns='YEAR', values='VALUE')
    
    all_years = set(pivot1.columns).union(set(pivot2.columns))
    if not all_years:
        print(f"Skipping {table_name}: No data for operands {inop} and {inri}")
        return
    if min_year is None:
        min_year = int(min(all_years))
    if max_year is None:
        max_year = int(max(all_years))
    full_years = list(range(min_year, max_year + 1))
    
    for year in full_years:
        if year not in pivot1.columns:
            pivot1[year] = np.nan
        if year not in pivot2.columns:
            pivot2[year] = np.nan
    pivot1 = pivot1[sorted(pivot1.columns)]
    pivot2 = pivot2[sorted(pivot2.columns)]

    opri = pivot1 / pivot2
    opri = country.merge(opri.reset_index(), on='COUNTRY_ID', how='left').drop(columns='COUNTRY_ID')
    opri = opri.rename(columns={'IFs Country': 'Country'})
    opri = opri.rename(columns={'FIPS':'FIPS_CODE'})
    for year in full_years:
        if year not in opri.columns:
            opri[year] = np.nan
    year_cols = sorted([col for col in opri.columns if str(col).isdigit()], key=int)
    opri = opri[['Country', 'FIPS_CODE'] + year_cols]
    opri['Earliest'] = opri[year_cols].bfill(axis=1).iloc[:, 0]
    opri['MostRecent'] = opri[year_cols].ffill(axis=1).iloc[:, -1]
    opri[year_cols + ['Earliest', 'MostRecent']] = opri[year_cols + ['Earliest', 'MostRecent']].round(dec_places)
    
    columns_sql = ['Country VARCHAR(255)', 'FIPS_CODE VARCHAR(255)'] + [f'"{col}" DOUBLE(53)' for col in year_cols + ['Earliest', 'MostRecent']]
    cursor.execute(f'DROP TABLE IF EXISTS "{table_name}"')
    cursor.execute(f"CREATE TABLE '{table_name}' ({', '.join(columns_sql)})")
    opri.to_sql(table_name, conn, if_exists='append', index=False)
    
tasks = [
    ('20062', '20162', 'SeriesPupilTeacherRatioUISPrimary'),
    ('20062', '20162', 'SeriesEDPriPTR'),
    ('20082', '20182', 'SeriesPupilTeacherRatioUISSecondary'),
    ('X.USCONST.02.FSGOV', 'X.USCONST.FSGOV', 'SeriesEdExpPrePri%EducTot'),
    ('X.USCONST.1.FSGOV', 'X.USCONST.FSGOV', 'SeriesEdExpPri%EducTot'),
    ('X.USCONST.2.FSGOV', 'X.USCONST.FSGOV', 'SeriesEdExpSecLowr%EducTot'),
    ('X.USCONST.2T3.FSGOV', 'X.USCONST.FSGOV', 'SeriesEdExpSec%EducTot'),
    ('20062', '20162', 'SeriesEdExpSecUppr%EducTot'),
    ('X.USCONST.5T8.FSGOV', 'X.USCONST.FSGOV', 'SeriesEdExpTer%EducTot'),
    ('20064', '20164', 'SeriesEdSecLowrPTR')
]
for op, ri, table in tasks:
    process_ratio(op, ri, table)

Adding

In [132]:
df = pd.read_csv(r'C:\Users\Norah\Downloads\OPRI (1)\OPRI_DATA_NATIONAL.csv', low_memory=False)
conn = sqlite3.connect(r'C:\IFs\RUNFILES\IFsDataImport - Copy (5) - Copy.db')
cursor = conn.cursor()

def process_indicators(indicator_ids, table_name, min_year=None, max_year=None):
    dfs = []
    for indicator in indicator_ids:
        dfa = df[df['INDICATOR_ID'] == indicator].drop(columns=['INDICATOR_ID', 'MAGNITUDE', 'QUALIFIER'], errors='ignore')
        if dfa.empty:
            continue
        pivoted = dfa.pivot(index='COUNTRY_ID', columns='YEAR', values='VALUE')
        dfs.append(pivoted)
    
    all_years = set()
    for d in dfs:
        all_years.update(d.columns)

    if not all_years:
        print(f"Skipping {table_name}: No data for indicators {indicator_ids}")
        return  

    if min_year is None:
        min_year = int(min(all_years))
    if max_year is None:
        max_year = int(max(all_years))
    full_years = list(range(min_year, max_year + 1)) 
    
    dec_places = datadict.loc[datadict['Variable'] == table_name, 'Decimal Places']
    dec_places = int(dec_places.iloc[0]) if not dec_places.empty and pd.notnull(dec_places.iloc[0]) else 4

    oprisum = sum(dfs)
    oprisum = country.merge(oprisum.reset_index(), on='COUNTRY_ID', how='left').drop(columns='COUNTRY_ID')
    oprisum = oprisum.rename(columns={'IFs Country': 'Country'})
    for year in full_years:
        if year not in oprisum.columns:
            oprisum[year] = np.nan

    year_columns = sorted([col for col in oprisum.columns if isinstance(col, int) or (isinstance(col, str) and str(col).isdigit())], key=lambda x: int(x))
    oprisum = oprisum[['Country', 'FIPS_CODE'] + year_columns]
    oprisum['Earliest'] = oprisum[year_columns].bfill(axis=1).iloc[:, 0]
    oprisum['MostRecent'] = oprisum[year_columns].ffill(axis=1).iloc[:, -1]
    oprisum[year_columns + ['Earliest', 'MostRecent']] = oprisum[year_columns + ['Earliest', 'MostRecent']].round(dec_places)

    columns_sql = ['Country VARCHAR(255)', 'FIPS_CODE VARCHAR(255)']
    columns_sql += [f'"{col}" DOUBLE(53)' for col in year_columns + ['Earliest', 'MostRecent']]
    create_table_sql = f"CREATE TABLE '{table_name}' ({', '.join(columns_sql)})"
    cursor.execute(f'DROP TABLE IF EXISTS "{table_name}"')
    cursor.execute(create_table_sql)
    oprisum.to_sql(table_name, conn, if_exists='append', index=False)

indicator_sets = {
    'SeriesGovtEdPub%GDP': ['XGDP.02.FSGOV', 'XGDP.1.FSGOV', 'XGDP.2.FSGOV', 'XGDP.3.FSGOV', 'XGDP.4.FSGOV', 'XGDP.5T8.FSGOV'],
    'SeriesEdExpISCED1-4TotalCur%': ['XSPENDP.1.FDPUB.FNCUR', 'XSPENDP.2.FDPUB.FNCUR', 'XSPENDP.3.FDPUB.FNCUR', 'XSPENDP.4.FDPUB.FNCUR'],
    'SeriesEdExpISCED1-4NonSalCur%': ['XSPENDP.1.FDPUB.FNNONS', 'XSPENDP.2.FDPUB.FNNONS', 'XSPENDP.3.FDPUB.FNNONS', 'XSPENDP.4.FDPUB.FNNONS'],
    'SeriesEdExpISCED1-4Sal%': ['XSPENDP.1.FDPUB.FNS', 'XSPENDP.2.FDPUB.FNS', 'XSPENDP.3.FDPUB.FNS', 'XSPENDP.4.FDPUB.FNS'],
    'SeriesEdExpISCED1-4Cap': ['XSPENDP.1.FDPUB.FNCAP', 'XSPENDP.2.FDPUB.FNCAP', 'XSPENDP.3.FDPUB.FNCAP', 'XSPENDP.4.FDPUB.FNCAP'],
    'SeriesEdTerGrads%Sci': ['FOSGP.5T8.F500', 'FOSGP.5T8.F600']
}

for table_name, indicators in indicator_sets.items():
    process_indicators(indicators, table_name)


DataDict

In [139]:
def safe_int_convert(val):
    if pd.isna(val):
        return val
    if isinstance(val, int) and abs(val) > 2**63 - 1:
        return str(val)
    return val
datadict = datadict.map(safe_int_convert)

conn = sqlite3.connect(r'C:\IFs\RUNFILES\IFsDataImport - Copy (5) - Copy.db')
cursor = conn.cursor()
table_name = 'DataDict'
datadict.to_sql(table_name, conn, if_exists='append', index=False)
conn.commit()
conn.close()